In [21]:
from functools import cached_property, reduce
from itertools import product as iterprod
import numpy as np

In [297]:
diagmats = [np.diag([1,1,1]), np.diag([1,-1,-1]), np.diag([-1,1,-1]), np.diag([-1,-1,1])]
cyclmats = [np.diag([1,1,1]),
        np.array([[0,-1,0],[1,0,0],[0,0,1]]),
        np.array([[1,0,0],[0,0,-1],[0,1,0]]),
        np.array([[0,0,-1],[0,1,0],[1,0,0]]),
        np.array([[0,0,1],[1,0,0],[0,1,0]]),
        np.array([[0,1,0],[0,0,1],[1,0,0]])
       ]

rotations = [a@b for a, b in iterprod(diagmats,cyclmats)]

def find_rotation(v1, v2):
    for k, r in enumerate(rotations):
        if np.all(v1 == r@v2):
            return k, r
    else:
        raise ValueError("The two vectors are not related")

class Scanner:
    def __init__(self, beacons, k=None):
        self.id = k
        self.b = np.asarray(beacons, dtype=int)
        self.rel_dist
        self.minb = 12 #minimum number of overlapping beacons to declare overlap
        if k == 0:
            self.s = np.array([0,0,0])
            self.rk = 0   
            
    @cached_property
    def rel_dist(self):
        return ((np.expand_dims(self.b,1)-np.expand_dims(self.b,0))**2).sum(axis=2)
    
    @property
    def nb(self):
        return self.b.shape[0]
    
    @property
    def r(self):
        return rotations[self.rk]
    
    def intersect(self, other):
        # in principle this is not enough: should count multiplicity
        return np.intersect1d(self.rel_dist.flatten(), other.rel_dist.flatten())[1:]   #remove first entry (0)
    
    def find_relation(self, other):
        common_dist = self.intersect(other) # relative distances between beacons measured by both self and other
        if len(common_dist) >= self.minb*(self.minb-1)//2:
            # match the common beacons
            common_i1 = reduce(np.union1d,(np.where(self.rel_dist == cd)[0] for cd in common_dist))
            common_i2 = reduce(np.union1d,(np.where(other.rel_dist == cd)[0] for cd in common_dist))
            beaconmap = []
            dset1 = [set(self.rel_dist[c][common_i1]) for c in common_i1]
            dset2 = [set(other.rel_dist[c][common_i2]) for c in common_i2]
            print("###find_relation###", len(common_dist), len(common_i1), len(common_i2), len(dset1), len(dset2))
            for ci1, drow1 in zip(common_i1, dset1):     # Can make it much shorter - I'm never using more than two! -need to return common_i2
                for ci2, drow2 in zip(common_i2, dset2):
                    if drow1 == drow2:
                        beaconmap.append([ci1,ci2])
            beaconmap = np.array(beaconmap)
            print(beaconmap)
            # find the relative distance and orientation of the scanners
            dself = self.b[beaconmap[0,0]]-self.b[beaconmap[1,0]]
            dother = other.b[beaconmap[0,1]]-other.b[beaconmap[1,1]]
            rot_id, r = find_rotation(dself, dother)
            sk = self.b[beaconmap[0,0]] - r @ other.b[beaconmap[0,1]] #used the first, they are all the same
            other.s = sk
            other.rk = rot_id
            return beaconmap
        else:
            raise ValueError("The two scanners do not have enough overlap")
        
    def lock(self, other):
        beaconmap = self.find_relation(other)

        mask = np.ones(other.nb, dtype=bool)
        mask[beaconmap[:,1]] = False   # mask to select the beacons of other that are not in self 
        
        new_c = other.s + other.b[mask] @ other.r.T
        self.b = np.vstack((self.b, new_c))
        
        # update distance matrix
        del self.rel_dist
        self.rel_dist

In [298]:
data = []

with open('day19.txt') as f:
    for l in f.readlines():
        if l[:3] == '---':
            scannerdata = []
        elif l == '\n':
            data.append(np.array(scannerdata, dtype=int))
        else:
            scannerdata.append(l.rstrip().split(','))

In [299]:
scanners = [Scanner(d, k) for k,d in enumerate(data)]

s = scanners.pop(0)

In [300]:
[q.id for q in scanners if len(s.intersect(q))>65]

[4, 19, 22, 30]

In [301]:
def find_overlapping():
    for k, q in enumerate(scanners):
        if len(s.intersect(q)) >= 66:
            return scanners.pop(k)

In [302]:
while len(scanners) > 0:
    print(f"{len(scanners)} scanners left, {s.nb} beacons found")
    q = find_overlapping()
    print(f"matched scanner #{q.id}")
    s.lock(q)

35 scanners left, 26 beacons found
matched scanner #4
###find_relation### 66 12 12 12 12
[[ 0  5]
 [ 2  3]
 [ 5 19]
 [ 6  7]
 [ 9 18]
 [11  1]
 [13  6]
 [16 21]
 [17 20]
 [20 16]
 [21 12]
 [23 23]]
34 scanners left, 39 beacons found
matched scanner #19
###find_relation### 66 12 12 12 12
[[ 0 12]
 [ 4 24]
 [ 5 21]
 [ 7 14]
 [ 8  7]
 [ 9  3]
 [12 15]
 [15  4]
 [16 10]
 [21 19]
 [22 20]
 [23  5]]
33 scanners left, 52 beacons found
matched scanner #10
###find_relation### 66 12 12 12 12
[[ 0  3]
 [ 5 16]
 [ 8 25]
 [12 23]
 [21 18]
 [22  1]
 [41 12]
 [42 20]
 [43  8]
 [45 17]
 [46  7]
 [51 22]]
32 scanners left, 66 beacons found
matched scanner #18
###find_relation### 66 12 12 12 12
[[ 4 21]
 [ 7 15]
 [ 8 24]
 [12 14]
 [15 11]
 [22 10]
 [39  2]
 [41 23]
 [44  8]
 [45 25]
 [50 16]
 [51 12]]
31 scanners left, 80 beacons found
matched scanner #1
###find_relation### 66 12 12 12 12
[[ 4 15]
 [ 7 21]
 [15  3]
 [39  0]
 [44 22]
 [50 12]
 [69 14]
 [70  9]
 [71 23]
 [72  6]
 [74 16]
 [75  5]]
30 scan

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [268]:
s.intersect(q)

array([   4941,    5102,    7765,   12850,   18386,   20901,   23417,
         23835,   24253,   26138,   28361,   30541,   32083,   36186,
         40453,  595974,  601492,  608009,  625001,  629763,  642386,
        663350,  669342,  679371, 1105577, 1105684, 1132309, 1153994,
       1258080, 1310950, 1319045, 1323582, 1378242, 1380107, 1436330,
       1454281, 1463810, 1476881, 1480520, 1485630, 1513357, 1560347,
       1643905, 1649555, 1655361, 1783377, 1812293, 1856904, 1876118,
       1956707, 1993741, 2026074, 2062133, 2127989, 2160093, 2169686,
       2178637, 2203499, 2245302, 2259646, 2277749, 2297294, 2357534,
       2368953, 2369574, 2382728, 2403461, 2439230, 2455124, 2483070,
       2488923, 2515629, 2550157, 2550969, 2575979, 2595866, 2661194,
       2666213, 2667317, 2779714, 2840538, 3190985, 3357758, 3377409,
       3429501, 3731229, 3763894, 3823369, 3981938, 3992781, 4016714,
       4054074, 4223403, 4328138, 4391643, 4552910, 4576017, 4904538,
       5161502, 5218